In [ ]:
#pip install pandas
#pip install seaborn


In [8]:
#pandas, sklearn, etc. 
import pandas as pd
import seaborn as sns 

Matplotlib is building the font cache; this may take a moment.


In [13]:
#load in csv

df = pd.read_csv('/Users/kristinachan/Analysis/2024-08-02_longcovid_interstate/LC_household_pulse_for_BV_8.csv')
# df.filter(like='leave').head(2)
df.head(2)

#option to make the index the state, may be more useful as the column for now

,State,Jun 24 Value,Jun 24 Low CI,Jun 24 High CI,Urban,Rural,Conservative,Moderate,Liberal,Political Don't know,...,9/4/23,10/2/23,10/30/23,2/5/24,3/4/24,4/1/24,4/29/24,5/27/24,6/24/24,Avg
0,Alabama,5.0,3.4,7.2,57.7,42.3,50.0,29.0,12.0,8.0,...,6.3,8.8,6.8,7.9,8.3,7.4,8.3,8.6,5.0,7.6
1,Alaska,6.7,4.3,10.0,64.9,35.1,34.0,37.0,23.0,6.0,...,9.1,4.5,5.4,9.2,7.7,8.2,5.7,6.8,6.7,7.0


In [ ]:
#you want to average the values of 1) paid leave and 2)unpaid leave separately
#legend: color by -- subgroup of paid & unpaid
#x-axis: longitudinal values in increasing order
#y-axis: average rates of long covid
#points: plot the average value for that date and subgroup
#line graph 

#next steps after that: scatterplot with linear regression 
    #q: do i care about the correlation between time and increase/decreasing covid rank?... 
#can also visualize with boxplots to see if trends are more apparent


In [ ]:
#scatterplots, etc